changed to markdown so that this code isn't run
from qsurface.main import initialize, run
%matplotlib tk

code, decoder = initialize((6,6), "planar", "mwpm", enabled_errors=["pauli"], faulty_measurements=True, plotting=True, initial_states=(0,0))
run(code, decoder, iteration=10, error_rates = {"p_bitflip": 0.05, "p_bitflip_plaq": 0.05}, decode_initial=False)

changed to markdown
from qsurface.main import initialize, run
code, decoder = initialize((6,6), "toric", "mwpm", enabled_errors=["pauli"])
run(code, decoder, iterations=10, error_rates = {"p_bitflip": 0.1})

Notes for self:


In [1]:
from qsurface.errors.pauli import Sim as PauliSim
from qsurface.codes.elements import DataQubit
from qsurface.codes.elements import AncillaQubit
from qsurface.codes.elements import Edge
from qsurface.main import initialize

In [30]:
error = PauliSim(p_bitflip = 0.05)

In [14]:
dqubit = DataQubit((0,0))

In [32]:
ancilla_a = AncillaQubit((0,0.5), state_type = "x")
ancilla_b = AncillaQubit((-0.5,0), state_type = "z")
ancilla_c = AncillaQubit((0.5,0), state_type = "z")
ancilla_d = AncillaQubit((0,-0.5), state_type = "x")

In [33]:
# see entangle_pair under qsurface.codes._template

ancilla_a.parity_qubits[(0,0)] = dqubit
ancilla_b.parity_qubits[(0,0)] = dqubit
ancilla_c.parity_qubits[(0,0)] = dqubit
ancilla_d.parity_qubits[(0,0)] = dqubit

x_edge = Edge(dqubit, state_type="x", initial_state=False)
z_edge = Edge(dqubit, state_type="z", initial_state=False)

dqubit.edges["x"] = x_edge
dqubit.edges["z"] = z_edge

x_edge.add_node(ancilla_a)
x_edge.add_node(ancilla_d)
z_edge.add_node(ancilla_b)
z_edge.add_node(ancilla_c)

In [34]:
dqubit.state

{'x': False, 'z': False}

In [35]:
error.random_error(dqubit, p_bitflip=0, p_phaseflip=1)

In [38]:
dqubit.state

{'x': True, 'z': True}

In [20]:
ancilla_a.measure()
ancilla_b.measure()
ancilla_c.measure()
ancilla_d.measure()

False

In [21]:
ancilla_a.syndrome

False

In [22]:
ancilla_d.syndrome

False

In [23]:
ancilla_b.syndrome

True

In [24]:
ancilla_c.syndrome

True

In [11]:
code, decoder = initialize((6,6), "planar", "mwpm", enabled_errors=["pauli"]) # can add ` "p_bitflip"=

In [12]:
code.logical_state

{'x': 1, 'z': 0}

In [13]:
# with some adjustments, this could be changed to applying random errors
def apply_surface_error(code, errors):
    # errors is a dictionary with the qubit position as key and error type as value
    # currently supported errors are Pauli (I, X, Y, or Z)
    # currently only works on data qubits
    # assumes qubits are on layer 0
    for qubit_pos in errors.keys():
        if errors[qubit_pos] == 'X':
            code.errors['pauli'].bitflip(code.data_qubits[0][qubit_pos])
        elif errors[qubit_pos] == 'Z':
            code.errors['pauli'].phaseflip(code.data_qubits[0][qubit_pos])
        elif errors[qubit_pos] == 'Y':
            code.errors['pauli'].bitphaseflip(code.data_qubits[0][qubit_pos])
    code.logical_state

In [17]:
error_dict = {(1.5, 0): "X"}

In [18]:
apply_surface_error(code, error_dict)

In [19]:
code.logical_state

{'x': 0, 'z': 0}

In [20]:
code.ancilla_qubits[0][(1,0)].syndrome

False

- get all syndromes
- Tanner graph (slides reference)